In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


In [4]:
system"l init.q"

//subsequent calls to this init will throw an error because we have changed directory 
    //this error can be safely ignored 

//if you need to do a hard reset, please restart the kernel. 

Database present - loading local partitioned database /home/jovyan/course-advanced/.hidden/db/taq


**Learning Outcomes**

To understand:
* Keyed Joins - uj,ej, pj
* Bitemporal (asof) joins 
* How to use aj 
* How to use wj 

# Introduction 

In a conventional database [joins](https://code.kx.com/q/basics/joins/) are primarily associated with tables, in which a join is used to extract data from a lookup table based on a common column or key. In q, tables are first-class entities in the language. You can define tables and relations statically, but it is easy to create them dynamically. It is even possible to join tables that could have a relation but do not.

## Keyed Joins

Keyed joins such as [```lj```](https://code.kx.com/q/ref/lj/) and [```ij```](https://code.kx.com/q/ref/ij/) can be used when tables need to be joined by specific field such as date or sym. Here is a list of keyed joins in q that were covered in KX Fundamental course:
* [Left Join - lj](https://code.kx.com/q/ref/lj/)
* [Inner Join - ij](https://code.kx.com/q/ref/ij/)

The following joins will be discussed below:

* [Plus Join - pj](https://code.kx.com/q/ref/pj/)
* [Union join - uj](https://code.kx.com/q/ref/uj/)  
* [Equi join - ej](https://code.kx.com/q/ref/ej/)

###### Recap

* **Left Join** -`lj` is most commonly used to supplement a time-series table with reference data - this appends new columns to our time-series LHS table. The `lj` is one of the most frequently used joins in kdb+/q. 

The syntax is:```<time-series table> lj <reference (keyed) table>```

In [5]:
timeSeriesTable:select date,time,sym, price,size from trade where date=2020.01.02 
refTable:`date xkey select date,time,bid, ask from quote where date=2020.01.02

timeSeriesTable lj refTable

date       time         sym  price size bid   ask  
---------------------------------------------------
2020.01.02 09:30:00.000 AAPL 83.88 17   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.87 74   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.84 57   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.87 81   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.87 52   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.83 20   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.98 67   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.97 47   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.95 70   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.9  62   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.94 18   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.89 32   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.85 72   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.76 10   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.77 25   83.19 84.19
..


* **Inner join** - `ij` operator acts in the same way as the left join, but only records for which the key matches between tables will remain in the result table. This appends new columns to our time-series LHS table, and removes non-common rows. 

The syntax is: ``LHSTable ij RHSTable``

In [6]:
timeSeriesTable ij refTable

date       time         sym  price size bid   ask  
---------------------------------------------------
2020.01.02 09:30:00.000 AAPL 83.88 17   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.87 74   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.84 57   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.87 81   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.87 52   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.83 20   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.98 67   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.97 47   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.95 70   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.9  62   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.94 18   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.89 32   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.85 72   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.76 10   83.19 84.19
2020.01.02 09:30:00.000 AAPL 83.77 25   83.19 84.19
..


### Plus Join `pj`

The [plus join](https://code.kx.com/q/ref/pj/) is used to sum matching columns of two tables. The left table can be either keyed or unkeyed and the right table must be keyed. The resulting table is all of the rows of the left table summed with the corresponding rows of the right table.

The Syntax: 

    pj[unkeyed or keyed;keyed]

Similar to `lj` the match is determined based on the key column in the secondary, or RHS table. 

In [10]:
show stocks:([]sym:`IBM`AAPL`GOOG;amount:400 700 1200)
show newpurchases:([sym:`IBM`GOOG]amount:60 30)
pj[stocks;newpurchases]

sym  amount
-----------
IBM  460   
AAPL 700   
GOOG 1230  
sym  amount
-----------
IBM  400   
AAPL 700   
GOOG 1200  
sym | amount
----| ------
IBM | 60    
GOOG| 30    


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Missing and null values are treated as zero</i></p>

In [11]:
newpurchases:([sym:`IBM`AAPL`GOOG]amount:60 0 30)
pj[stocks;newpurchases] //same result as above

sym  amount
-----------
IBM  460   
AAPL 700   
GOOG 1230  


### Union Join `uj`
[Union join](https://code.kx.com/q/ref/uj/) appends data as rows in contrast to `ij` and `lj` - columns will be added where they are not already present to create a combined schema between the two tables. 

The syntax: 

     LHSTable uj RHSTable

This is the most flexible join and when both tables are unkeyed looks to create a combined schema between the two tables that will allow all records to be combined. This will not throw an error in the event of type conflicts between common columns.

In [12]:
//creating tables
show smallTrade:([]time:09:00+10*til 5;sym:`JPM`GE`JPM`IBM`GE;price:30+5?3.;size:5?20) 
//created a key table keyed on sym
show reference:([sym:`JPM`IBM]companyName:`$("JP Morgan";"International Business Machines");sector:`Banking`IT) 

time  sym price    size
-----------------------
09:00 JPM 30.31721 0   
09:10 GE  30.31348 13  
09:20 JPM 30.09401 11  
09:30 IBM 30.48213 16  
09:40 GE  30.42529 11  
sym| companyName                     sector 
---| ---------------------------------------
JPM| JP Morgan                       Banking
IBM| International Business Machines IT     


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Remember you can unkey a table using `0!`</i></p>

In [13]:
smallTrade uj 0! reference                   //common schema used and data for each present as separate rows
smallTrade uj ([]size: 2 3f),'0! reference   //adding a size column to reference with type float

time  sym price    size companyName                     sector 
---------------------------------------------------------------
09:00 JPM 30.31721 0                                           
09:10 GE  30.31348 13                                          
09:20 JPM 30.09401 11                                          
09:30 IBM 30.48213 16                                          
09:40 GE  30.42529 11                                          
      JPM               JP Morgan                       Banking
      IBM               International Business Machines IT     
time  sym price    size companyName                     sector 
---------------------------------------------------------------
09:00 JPM 30.31721 0                                           
09:10 GE  30.31348 13                                          
09:20 JPM 30.09401 11                                          
09:30 IBM 30.48213 16                                          
09:40 GE  30.42529 11                   

In the event of just one table being keyed, an error is thrown. If both tables are keyed on the same column we get the corresponding records in each table for the set of keys :

In [14]:
(`sym xkey smallTrade) uj reference 
reference uj (`sym xkey smallTrade)

sym| time  price    size companyName                     sector 
---| -----------------------------------------------------------
JPM| 09:00 30.31721 0    JP Morgan                       Banking
GE | 09:10 30.31348 13                                          
IBM| 09:30 30.48213 16   International Business Machines IT     
sym| companyName                     sector  time  price    size
---| -----------------------------------------------------------
JPM| JP Morgan                       Banking 09:00 30.31721 0   
IBM| International Business Machines IT      09:30 30.48213 16  
GE |                                         09:10 30.31348 13  


###### Example
One use of `uj` could be to time order data from two different tables in order to ascertain the sequence of updates across multiple tables. For example, trade and order tables could be joined as below:

In [19]:
show nums: 25?.Q.n
5 cut nums

"64578"
"04629"
"59722"
"95169"
"89269"
"6457804629597229516989269"


In [20]:
smallTrade // Using the trade table defined above
/creating an order table
show order:([]time:asc `minute$08:00+5?0D02:00:00;
             sym:5?`JPM`GE`IBM;
             orderID:(5 cut 25?.Q.n),'string 5?`5;
             price:30+5?20.)

time  sym price    size
-----------------------
09:00 JPM 30.31721 0   
09:10 GE  30.31348 13  
09:20 JPM 30.09401 11  
09:30 IBM 30.48213 16  
09:40 GE  30.42529 11  
time  sym orderID      price   
-------------------------------
08:13 JPM "82270kmhjm" 38.72898
08:15 GE  "42232fhmoo" 40.5766 
08:37 JPM "68303aemab" 36.70438
08:40 JPM "71190cpejg" 39.89686
09:00 GE  "61242npklc" 30.45133


It is now easy to see the sequence in which trades and orders happened.

In [21]:
`time xasc uj[smallTrade;order]

time  sym price    size orderID     
------------------------------------
08:13 JPM 38.72898      "82270kmhjm"
08:15 GE  40.5766       "42232fhmoo"
08:37 JPM 36.70438      "68303aemab"
08:40 JPM 39.89686      "71190cpejg"
09:00 JPM 30.31721 0    ""          
09:00 GE  30.45133      "61242npklc"
09:10 GE  30.31348 13   ""          
09:20 JPM 30.09401 11   ""          
09:30 IBM 30.48213 16   ""          
09:40 GE  30.42529 11   ""          


### Equi Join `ej`
[Equi join](https://code.kx.com/q/ref/ej/) joins two tables on specified column(s). The result is a table with all of the rows in the first table that match the second table on the specified columns.

The Syntax: 
    
    ej[matchingColumns;table1;table2]

Using the trade and reference tables that were defined above, we can see that `ej` returns both `JPM` rows: 

In [22]:
smallTrade

time  sym price    size
-----------------------
09:00 JPM 30.31721 0   
09:10 GE  30.31348 13  
09:20 JPM 30.09401 11  
09:30 IBM 30.48213 16  
09:40 GE  30.42529 11  


In [23]:
reference

sym| companyName                     sector 
---| ---------------------------------------
JPM| JP Morgan                       Banking
IBM| International Business Machines IT     


In [24]:
ej[`sym;smallTrade;0!reference]

time  sym price    size companyName                     sector 
---------------------------------------------------------------
09:00 JPM 30.31721 0    JP Morgan                       Banking
09:20 JPM 30.09401 11   JP Morgan                       Banking
09:30 IBM 30.48213 16   International Business Machines IT     


In [25]:
(`sym xkey smallTrade) uj reference  //comparing to union join above

sym| time  price    size companyName                     sector 
---| -----------------------------------------------------------
JPM| 09:00 30.31721 0    JP Morgan                       Banking
GE | 09:10 30.31348 13                                          
IBM| 09:30 30.48213 16   International Business Machines IT     


##### Example 

Let's say we want to find all the trades that occurred at the full quote size, we can join the trade and quote tables with `ej`.  

In [26]:
dts:2020.01.02
t:select tradetime:time,sym, tprice:price,tsize:size from trade where date=dts 
q:select qtime:time,tprice:bid,tsize:bsize,sym from quote where date=dts

Looking at the bid side, we can find all the trade and quote times for all possible occurrence of a trade occurring at the bid price:

In [27]:
ej[`sym`tsize`tprice;t;q]

tradetime    sym  tprice tsize qtime       
-------------------------------------------
09:30:00.025 AAPL 83.87  74    15:10:00.637
09:30:00.028 AAPL 83.84  57    15:17:16.229
09:30:00.031 AAPL 83.87  81    09:30:00.356
09:30:00.031 AAPL 83.87  81    15:07:07.381
09:30:00.041 AAPL 83.87  52    14:25:24.482
09:30:00.147 AAPL 83.83  20    14:31:43.196
09:30:00.216 AAPL 83.98  67    15:35:26.500
09:30:00.441 AAPL 83.9   62    09:35:09.790
09:30:00.441 AAPL 83.9   62    12:22:08.045
09:30:00.536 AAPL 83.94  18    09:31:33.810
09:30:00.536 AAPL 83.94  18    12:24:25.832
09:30:00.575 AAPL 83.89  32    14:28:50.318
09:30:00.594 AAPL 83.85  72    09:31:03.923
09:30:00.594 AAPL 83.85  72    09:50:10.329
09:30:00.796 AAPL 83.77  25    12:39:17.476
..


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> <code>lj</code> and <code>ij</code> are much faster than the equivalent <code>ej</code> joins</i></p>

##### Quiz Time!
Try Exercise 1.1 to test your keyed joins knowledge

##### Exercise 1.1

Create a table journey with the fields Start, End, and Price, filled with 10 entries. Start should be populated with a random selection from Waterloo, Paddington and Euston. End and Price should be appropriately set.

In [ ]:
Journeys:([]Start:10?`Waterloo`Paddington`Euston;End:10?`Bristol`Cardiff`Glasgow`Birmingham;Price:10?30.0)

In [34]:
//Your answers here
start:10?`Waterloo`Paddington`Euston
end:10?`Cambidge`London`Liverpool
price:10?10+til 40
show table1:([]start;end;price)

start      end       price
--------------------------
Euston     Cambidge  42   
Waterloo   London    14   
Paddington Cambidge  23   
Waterloo   Cambidge  19   
Paddington London    45   
Euston     Cambidge  29   
Euston     Cambidge  28   
Waterloo   Liverpool 45   
Euston     Liverpool 41   
Euston     London    46   


Define a table:

```
Transport:([]Station:`Waterloo`Paddington`Euston;Direction:`South`West`North;Company:`Southwest`GNER`Virgin;TrainColour:`Blue`Green`Red)```

Using a suitable type of join, link transport to journey so that we have Start, End, Price, Company 

In [ ]:
Transport:([]Station:`Waterloo`Paddington`Euston;Direction:`South`West`North;Company:`Southwest`GNER`Virgin;TrainColour:`Blue`Green`Red)
Journeys lj (`Start xkey select Start:Station, Company from Transport)

In [41]:
//Your answers here
//show Transport:([]Station:`Waterloo`Paddington`Euston;Direction:`South`West`North;Company:`Southwest`GNER`Virgin;TrainColour:`Blue`Green`Red)
show compTable:table1 lj (`start xkey select start:Station, Company from Transport)

start      end       price Company  
------------------------------------
Euston     Cambidge  42    Virgin   
Waterloo   London    14    Southwest
Paddington Cambidge  23    GNER     
Waterloo   Cambidge  19    Southwest
Paddington London    45    GNER     
Euston     Cambidge  29    Virgin   
Euston     Cambidge  28    Virgin   
Waterloo   Liverpool 45    Southwest
Euston     Liverpool 41    Virgin   
Euston     London    46    Virgin   


Southwest decides to start 2 new services from Euston to Glasgow and Bristol priced at 21.1 and 3.14 respectively. Make a table NewJourneys with the same schema as the table above (i.e. Start, End, Price, Company) to reflect this. Then join this table to the table created above.

In [ ]:
NewJourneys:([]Start:`Euston`Euston;End:`Glasgow`Bristol;Price:21.1 3.14;Company:`Southwest`Southwest)
NewJourneys uj Journeys lj (`Start xkey select Start:Station, Company from Transport)

In [51]:
//Your answers here
newTable:([]start:`Euston`Euston;end:`Glasgow`Bristol;price: 21.1 3.14;Company:`SouthWest`SouthWest)
compTable,newTable //or uj

start      end       price Company  
------------------------------------
Euston     Cambidge  42    Virgin   
Waterloo   London    14    Southwest
Paddington Cambidge  23    GNER     
Waterloo   Cambidge  19    Southwest
Paddington London    45    GNER     
Euston     Cambidge  29    Virgin   
Euston     Cambidge  28    Virgin   
Waterloo   Liverpool 45    Southwest
Euston     Liverpool 41    Virgin   
Euston     London    46    Virgin   
Euston     Glasgow   21.1  SouthWest
Euston     Bristol   3.14  SouthWest


In [44]:
meta newTable

c      | t f a
-------| -----
start  | s    
end    | s    
price  | f    
company| s    


In [45]:
meta compTable

c      | t f a
-------| -----
start  | s    
end    | s    
price  | j    
Company| s    


# Bitemporal joins (aka "asof" joins)
Some joins are [bitemporal](https://en.wikipedia.org/wiki/Temporal_database), meaning they take into account the time in two dimensions, or for us, as per two different tables. 

Below is a list of bitemporal joins:
* [asof](https://code.kx.com/q/ref/aj/) - combines data based as per the time in one table with the *prevailing* values in another table. 
* [window-join](https://code.kx.com/v2/ref/wj/) - combines data based as per the time in one table with the values within a specified window in another table. 

## Asof Join 

As the name may suggest [Asof join](https://code.kx.com/q/ref/aj/) is mainly used to join columns with reference to time.
 

The Syntax 

    aj[col1,…,coln,timeCol;source table;reference table]

Where `col1` to `coln` are exact match columns and `timeCol` is the common time column that should be used for the time lookup. An `aj` also assumes that the time in our reference table is sorted - if it is not we will get odd results. 

It might be useful to see an example to better understand this concept, below we will create a quote and trade table:

Using the asof join, we will be able to determine the prevailing quote for each trade.

In [52]:
show q:([]time:09:29 09:29 09:32 09:33;sym:`JPM`AAPL`JPM`AAPL;ask:30.23 40.20 30.35 40.35;bid:30.20 40.19 30.33 40.32)

show t:([]time:09:30 09:31 09:32 09:33 09:34 09:35;sym:`JPM`AAPL`AAPL`JPM`AAPL`JPM;price:30.43 30.45 40.45 30.55 41.00 31.00;size:100 200 200 300 300 600)

time  sym  ask   bid  
----------------------
09:29 JPM  30.23 30.2 
09:29 AAPL 40.2  40.19
09:32 JPM  30.35 30.33
09:33 AAPL 40.35 40.32
time  sym  price size
---------------------
09:30 JPM  30.43 100 
09:31 AAPL 30.45 200 
09:32 AAPL 40.45 200 
09:33 JPM  30.55 300 
09:34 AAPL 41    300 
09:35 JPM  31    600 


In [53]:
aj[`sym`time;t;q]

time  sym  price size ask   bid  
---------------------------------
09:30 JPM  30.43 100  30.23 30.2 
09:31 AAPL 30.45 200  40.2  40.19
09:32 AAPL 40.45 200  40.2  40.19
09:33 JPM  30.55 300  30.35 30.33
09:34 AAPL 41    300  40.35 40.32
09:35 JPM  31    600  30.35 30.33


We can see that only the trades quotes corresponding to the same `sym` where used, and the last previous value as per the quote table was used. 

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Don't forget <code>aj</code> assumes the time column in our reference table is sorted in ascending order!</i></p>

### How to use ```aj```? 
What happens if we switch the order of the matching columns around ?

In [56]:
aj[`time`sym;t;q]

time  sym  price size ask   bid  
---------------------------------
09:30 JPM  30.43 100             
09:31 AAPL 30.45 200             
09:32 AAPL 40.45 200             
09:33 JPM  30.55 300  40.35 40.32
09:34 AAPL 41    300             
09:35 JPM  31    600             


As we can see, it's giving us the wrong answer. The last specified join column must be a temporal column. Asof joins uses a binary search algorithm to find the matching rows. The non-temporal specified join column is used to find the exact matching rows before performing the binary search. 

As you can see from the above result, there is only time column associated with the trade table - is it possible to retrieve the time column from the quote table ?   

In [57]:
aj0[`sym`time;t;q] //using aj0 

time  sym  price size ask   bid  
---------------------------------
09:29 JPM  30.43 100  30.23 30.2 
09:29 AAPL 30.45 200  40.2  40.19
09:29 AAPL 40.45 200  40.2  40.19
09:32 JPM  30.55 300  30.35 30.33
09:33 AAPL 41    300  40.35 40.32
09:32 JPM  31    600  30.35 30.33


Is it possible to retrieve the time column from both trade and quote?

In [58]:
aj[`sym`time;t;select time,qtime:time,sym,ask, bid from q] //adding a copy of time column 

time  sym  price size qtime ask   bid  
---------------------------------------
09:30 JPM  30.43 100  09:29 30.23 30.2 
09:31 AAPL 30.45 200  09:29 40.2  40.19
09:32 AAPL 40.45 200  09:29 40.2  40.19
09:33 JPM  30.55 300  09:32 30.35 30.33
09:34 AAPL 41    300  09:33 40.35 40.32
09:35 JPM  31    600  09:32 30.35 30.33


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Using this method eliminates the confusion of what time is returned.</i></p>

When joining two large tables, performance is always one thing to watch out for. The asof join is very sensitive to how fast the exact match can be found. Matching on more than two columns can impact the performance: 

In [59]:
count trade
count quote
multiDayTrade:select from trade where date within (2020.01.02;2020.01.04),time within (09:00;10:00)
multiDayQuote:select from quote where date within (2020.01.02;2020.01.04),time within (09:00;10:00)

3414935
16336312


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>In this instance, we are using on-disk tables so we will have to map them into memory using <code>select ... from trade </code> </i></p>

When you need to perform an asof join with 3 or more columns, it is more efficient to join each date individually and merge the result. The two reasons are:
* There is no need to load in quotes for multiple days into memory. 
* The attributes within each quote partition are preserved. 


In [60]:
g:multiDayTrade group multiDayTrade[`date] 
raze {aj[`sym`time;y]select from multiDayQuote where date=x}'[key g;g]

date       sym  time         price size stop cond ex bid   ask   bsize asize mode
---------------------------------------------------------------------------------
2020.01.02 AAPL 09:30:00.021 83.88 17   0    G    N  83.28 83.99 63    12    O   
2020.01.02 AAPL 09:30:00.025 83.87 74   0    J    N  83.43 84.75 53    33    Z   
2020.01.02 AAPL 09:30:00.028 83.84 57   0    N    N  83.08 84.51 88    62    Y   
2020.01.02 AAPL 09:30:00.031 83.87 81   0    K    N  83.08 84.14 15    42    R   
2020.01.02 AAPL 09:30:00.041 83.87 52   0    G    N  83.18 84.42 43    57    A   
2020.01.02 AAPL 09:30:00.147 83.83 20   0    Z    N  83.48 84.54 14    21    H   
2020.01.02 AAPL 09:30:00.216 83.98 67   0    8    N  83.91 84.67 50    63    O   
2020.01.02 AAPL 09:30:00.413 83.97 47   0    P    N  83.54 84.66 83    63    Z   
2020.01.02 AAPL 09:30:00.439 83.95 70   0    8    N  83.29 84.88 55    83    N   
2020.01.02 AAPL 09:30:00.441 83.9  62   0    A    N  83.44 84.26 31    42    L   
2020.01.02 AAPL 

[```asof```]() performs the same match as `aj` however we match it against a single record. It removes the columns that we used to match it with. Below we will see that the result table only consists of price and size.

In [61]:
t asof `sym`time!(`JPM;09:32)
t asof ([]sym:`JPM`AAPL;time:09:31 09:35)

price| 30.43
size | 100
price size
----------
30.43 100 
41    300 


##### Quiz Time!
Now that we've covered Bitemporal Joins, why not try Execrise 1.2 to test your knowledge! 

##### Exercise 1.2

Using the covid table below, create a table that contains the number of cases as of 3rd,10th and 20th of April 2020 for the UK, Spain and Italy. 

```
country date       noOfCases
----------------------------
Ireland 2020.04.01 3849     
UK      2020.04.02 33718    
Spain   2020.04.03 102316   
Italy   2020.04.04 132527   
China   2020.04.05 82300    
US      2020.04.06 425800   
Ireland 2020.04.07 7698     
UK      2020.04.08 67436    
Spain   2020.04.09 204632   
Italy   2020.04.10 265054   
China   2020.04.11 164600   
US      2020.04.12 851600
```

The resulting table should look like:
```
date       country noOfCases
----------------------------
2020.04.03 UK      33718    
2020.04.10 Spain   204632   
2020.04.20 Italy   265054  
```

In [62]:
show covidtable:([]country:L,L:`Ireland`UK`Spain`Italy`China`US;date:2020.04.01+til 12;noOfCases:n,2*n:3849 33718 102316 132527 82300 425800)

country date       noOfCases
----------------------------
Ireland 2020.04.01 3849     
UK      2020.04.02 33718    
Spain   2020.04.03 102316   
Italy   2020.04.04 132527   
China   2020.04.05 82300    
US      2020.04.06 425800   
Ireland 2020.04.07 7698     
UK      2020.04.08 67436    
Spain   2020.04.09 204632   
Italy   2020.04.10 265054   
China   2020.04.11 164600   
US      2020.04.12 851600   


In [ ]:
events:([]date:2020.04.03 2020.04.10 2020.04.20;country:`UK`Spain`Italy)
aj[`country`date;events;covidtable]

In [63]:
//Your answers here
datesOfInterest:([]date:2020.04.03 2020.04.10 2020.04.20;country:`UK`Spain`Italy)
aj[`country`date;datesOfInterest;covidtable]

date       country noOfCases
----------------------------
2020.04.03 UK      33718    
2020.04.10 Spain   204632   
2020.04.20 Italy   265054   


## Window join

[Window join](https://code.kx.com/q/ref/wj/) is a generalization of as-of joins, and rather than retrieving just the last value are designed to aggregate values within certain intervals. 

The Syntax: 

    wj[windowS;columns;source table;(reference table;(function1;col1);(function2;col2)...)]

Where `windowS` is a two item list of `(startTimes;endTimes)`, each of the same length as the source tables. These items correspond to the `startTimes` and `endTimes` for each window for every row in our source table. The `columns` input is the same as we saw for `aj`, a series of exact match columns and the final common time column. 

### How to use ```wj```?

The syntax and explanations are more complicated that going through an example: 

In [64]:
show smalltrade:([]sym:3#`JPM;time:09:30:01 09:30:04 09:30:08;price:120 123 121)
show smallquote:([]sym:10#`JPM;time:asc 09:30:00+10?8;ask:10?90+til 20;bid:10?90+til 20)

sym time     price
------------------
JPM 09:30:01 120  
JPM 09:30:04 123  
JPM 09:30:08 121  
sym time     ask bid
--------------------
JPM 09:30:00 103 94 
JPM 09:30:01 96  95 
JPM 09:30:02 102 107
JPM 09:30:02 95  91 
JPM 09:30:02 106 106
JPM 09:30:04 95  104
JPM 09:30:04 109 103
JPM 09:30:05 92  109
JPM 09:30:05 101 102
JPM 09:30:05 108 99 


Let's construct a 3 second set of windows (2 seconds before and 1 second after each trade time) for each of our trades:

In [70]:
smalltrade[`time]                         //our trade times 
show windows:(smalltrade[`time]-2;smalltrade[`time]+1)  //(window start time;window end time)
//windows
//or using iterators to add -2 and +1 to each time:
//windows: -2 1+\: smalltrade[`time] 
//windows

09:30:01 09:30:04 09:30:08
09:29:59 09:30:02 09:30:06
09:30:02 09:30:05 09:30:09


In [77]:
count windows        //start time and end time 
count each windows   //have a start time and end time for each of our trades (each row of trades)
count smalltrade
type windows

2
3 3
3
0h


Using these windows, we can join t and q calculating the highest ask and lowest bid within the window.

In [71]:
wj[windows;`sym`time;smalltrade;(smallquote;(max;`ask);(min;`bid))]

sym time     price ask bid
--------------------------
JPM 09:30:01 120   106 91 
JPM 09:30:04 123   109 99 
JPM 09:30:08 121   108 99 


In some cases we might want the intermediate values returned - to do that we can use the functional null:  

In [72]:
t
q
wj[windows;`sym`time;smalltrade;(smallquote;(::;`ask);(::;`bid))]

time  sym  price size
---------------------
09:30 JPM  30.43 100 
09:31 AAPL 30.45 200 
09:32 AAPL 40.45 200 
09:33 JPM  30.55 300 
09:34 AAPL 41    300 
09:35 JPM  31    600 
time  sym  ask   bid  
----------------------
09:29 JPM  30.23 30.2 
09:29 AAPL 40.2  40.19
09:32 JPM  30.35 30.33
09:33 AAPL 40.35 40.32
sym time     price ask                   bid                   
---------------------------------------------------------------
JPM 09:30:01 120   103 96 102 95 106     94 95 107 91 106      
JPM 09:30:04 123   106 95 109 92 101 108 106 104 103 109 102 99
JPM 09:30:08 121   ,108                  ,99                   


It's important to notice that the number of records we have returned in each window is not necessarily the same - in fact usually these will differ. We can use our own lambdas in place of keywords also. 

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>When writing your own functions for use within a <code>wj</code> don't forget that this function should take as input a <b>list</b> of values.</i></p>

``wj`` takes the initial value within the window to be the prevailing values that existed before the window started. To consider the values that only occurred during the window, we could use [``wj1``](https://code.kx.com/q/ref/wj/): 

In [73]:
wj1[windows;`sym`time;smalltrade;(smallquote;(::;`ask);(::;`bid))]

sym time     price ask                          bid                          
-----------------------------------------------------------------------------
JPM 09:30:01 120   103 96 102 95 106            94 95 107 91 106             
JPM 09:30:04 123   102 95 106 95 109 92 101 108 107 91 106 104 103 109 102 99
JPM 09:30:08 121   `long$()                     `long$()                     


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> <code>wj</code> can only work with integer temporal values. Datetime and any float values can't be use with <code>wj</code> or <code>wj1</code>. </i></p>

##### Example 

Window joins are widely used when investigating the relationship between trades and quotes in finance. For example, if we wanted to see how well a trade was executed, we would need to check the range of bid and ask prices that were prevalent around the trade time.

Let's look at the 2nd of January 2020 for Apple in the database:

In [74]:
janTrade:select from trade where date=2020.01.02,sym=`AAPL
janQuote:select from quote where date=2020.01.02,sym=`AAPL

Let's create a fixed-width windows of 2 seconds before and one second after each trade time.

In [75]:
w:-2 1+\: janTrade[`time]
w

09:30:00.019 09:30:00.023 09:30:00.026 09:30:00.029 09:30:00.039 09:30:00.145 09:30:00.214 09:30:00.411 09:30:00.437 09:30:00.439 09:30:00.534 09:30:00.573 09:30:00.592 09:30:00.644 09:30:00.794 09..
09:30:00.022 09:30:00.026 09:30:00.029 09:30:00.032 09:30:00.042 09:30:00.148 09:30:00.217 09:30:00.414 09:30:00.440 09:30:00.442 09:30:00.537 09:30:00.576 09:30:00.595 09:30:00.647 09:30:00.797 09..


Now we apply `wj` to find the maximum ask and minimum bid over each window

In [76]:
wj[w;`sym`time;janTrade;(janQuote;(max;`ask);(min;`bid))]

date       sym  time         price size stop cond ex ask   bid  
----------------------------------------------------------------
2020.01.02 AAPL 09:30:00.021 83.88 17   0    G    N  84.45 82.99
2020.01.02 AAPL 09:30:00.025 83.87 74   0    J    N  84.75 82.93
2020.01.02 AAPL 09:30:00.028 83.84 57   0    N    N  84.75 82.97
2020.01.02 AAPL 09:30:00.031 83.87 81   0    K    N  84.14 83.08
2020.01.02 AAPL 09:30:00.041 83.87 52   0    G    N  84.69 83.18
2020.01.02 AAPL 09:30:00.147 83.83 20   0    Z    N  84.54 82.87
2020.01.02 AAPL 09:30:00.216 83.98 67   0    8    N  84.67 83.91
2020.01.02 AAPL 09:30:00.413 83.97 47   0    P    N  84.85 83.12
2020.01.02 AAPL 09:30:00.439 83.95 70   0    8    N  84.88 83.19
2020.01.02 AAPL 09:30:00.441 83.9  62   0    A    N  84.88 83.29
2020.01.02 AAPL 09:30:00.536 83.94 18   0    G    N  83.98 83.39
2020.01.02 AAPL 09:30:00.575 83.89 32   0    G    N  84.72 82.99
2020.01.02 AAPL 09:30:00.594 83.85 72   0    A    N  84.72 83.76
2020.01.02 AAPL 09:30:00.

# Summary of all joins in kdb+

Now that we have all joins in kdb+ covered, we can see how they behave differently below:
<img src="../images/summaryJoins.png" width="2000" height="600">